In [1]:
import re
import os
import jieba
import logging

import numpy as np
import pandas as pd

from gensim.models.word2vec import LineSentence, Word2Vec

In [2]:
# import os
# import pathlib

# 预处理数据 构建数据集
is_build_dataset = True

# 获取项目根目录
root = '/Users/wenjiazhai/Documents/学习资料/开课吧/企业课/project 1/codes'

# 训练数据路径
train_data_path = os.path.join(root, 'data', 'train.csv')
# 测试数据路径
test_data_path = os.path.join(root, 'data', 'test.csv')
# 停用词路径
# stop_word_path = os.path.join(root, 'data', 'stopwords/哈工大停用词表.txt')
stop_word_path = os.path.join(root, 'data', 'stopwords/stopwords.txt')

# 自定义切词表
user_dict = os.path.join(root, 'data', 'user_dict.txt')

# 0. 预处理
# 预处理后的训练数据
train_seg_path = os.path.join(root, 'data', 'train_seg_data.csv')
# 预处理后的测试数据
test_seg_path = os.path.join(root, 'data', 'test_seg_data.csv')
# 合并训练集测试集数据
merger_seg_path = os.path.join(root, 'data', 'merged_train_test_seg_data.csv')

# 1. 数据标签分离
train_x_seg_path = os.path.join(root, 'data', 'train_X_seg_data.csv')
train_y_seg_path = os.path.join(root, 'data', 'train_Y_seg_data.csv')

val_x_seg_path = os.path.join(root, 'data', 'val_X_seg_data.csv')
val_y_seg_path = os.path.join(root, 'data', 'val_Y_seg_data.csv')

test_x_seg_path = os.path.join(root, 'data', 'test_X_seg_data.csv')

# 2. pad oov处理后的数据
train_x_pad_path = os.path.join(root, 'data', 'train_X_pad_data.csv')
train_y_pad_path = os.path.join(root, 'data', 'train_Y_pad_data.csv')
test_x_pad_path = os.path.join(root, 'data', 'test_X_pad_data.csv')
test_y_pad_path = os.path.join(root, 'data', 'test_Y_pad_data.csv')

# 3. numpy 转换后的数据
train_x_path = os.path.join(root, 'data', 'train_X')
train_y_path = os.path.join(root, 'data', 'train_Y')
test_x_path = os.path.join(root, 'data', 'test_X')
test_y_path = os.path.join(root, 'data', 'test_Y')

# 词向量路径
save_wv_model_path = os.path.join(root, 'data', 'wv', 'word2vec.model')
# 词向量矩阵保存路径
embedding_matrix_path = os.path.join(root, 'data', 'wv', 'embedding_matrix')
# 字典路径
vocab_path = os.path.join(root, 'data', 'wv', 'vocab.txt')
reverse_vocab_path = os.path.join(root, 'data', 'wv', 'reverstest_save_dire_vocab.txt')

# 词向量训练轮数
wv_train_epochs = 5

# 模型保存文件夹
# checkpoint_dir = os.path.join(root, 'data', 'checkpoints', 'training_checkpoints_pgn_cov_not_clean')

checkpoint_dir = os.path.join(root, 'data', 'checkpoints', 'training_checkpoints_pgn_cov_backed')

# checkpoint_dir = os.path.join(root, 'data', 'checkpoints', 'training_checkpoints_seq2seq')
seq2seq_checkpoint_dir = os.path.join(root, 'data', 'checkpoints', 'training_checkpoints_seq2seq')

checkpoint_prefix = os.path.join(checkpoint_dir, 'ckpt')

# 结果保存文件夹
save_result_dir = os.path.join(root, 'result')

# 词向量维度
embedding_dim = 300

sample_total = 82871

batch_size = 32
# batch_size = 4

epochs = 10

vocab_size = 30000


In [ ]:
from multiprocessing import cpu_count, Pool

# cpu 数量
cores = cpu_count()
# 分块个数
partitions = cores


def parallelize(df, func):
    """多核并行处理模块
    :param df: DataFrame数据
    :param func: 预处理函数
    :return: 处理后的数据
    """
    # 数据切分
    data_split = np.array_split(df, partitions)
    # 线程池
    pool = Pool(cores)
    # 数据分发 合并
    data = pd.concat(pool.map(func, data_split))
    # 关闭线程池
    pool.close()
    # 执行完close后不会有新的进程加入到pool,join函数等待所有子进程结束
    pool.join()
    return data

def save_dict(save_path, dict_data):
    """保存字典
    :param save_path: 保存路径
    :param dict_data: 字典路径
    """
    with open(save_path, 'w', encoding='utf-8') as f:
        for k, v in dict_data.items():
            f.write("{}\t{}\n".format(k, v))
            


In [ ]:
import numpy as np
from gensim.models.word2vec import Word2Vec

class Vocab:
    PAD_TOKEN = '<PAD>'
    UNKNOWN_TOKEN = '<UNK>'
    START_DECODING = '<START>'
    STOP_DECODING = '<STOP>'
    MASKS = [PAD_TOKEN, UNKNOWN_TOKEN, START_DECODING, STOP_DECODING]
    MASK_COUNT = len(MASKS)

    PAD_TOKEN_INDEX = MASKS.index(PAD_TOKEN)
    UNKNOWN_TOKEN_INDEX = MASKS.index(UNKNOWN_TOKEN)
    START_DECODING_INDEX = MASKS.index(START_DECODING)
    STOP_DECODING_INDEX = MASKS.index(STOP_DECODING)

    def __init__(self, vocab_file=vocab_path, vocab_max_size=None):
        """Vocab 对象,vocab基本操作封装
        :param vocab_file: Vocab 存储路径
        :param vocab_max_size: 最大字典数量
        """
        self.word2id, self.id2word = self.load_vocab(vocab_file, vocab_max_size)
        self.count = len(self.word2id)

    @staticmethod
    def load_vocab(file_path, vocab_max_size=None):
        """读取字典
        :param file_path: 文件路径
        :param vocab_max_size:
        :return: 返回读取后的字典
        """
        vocab = {mask: index
                 for index, mask in enumerate(Vocab.MASKS)}

        reverse_vocab = {index: mask
                         for index, mask in enumerate(Vocab.MASKS)}

        for line in open(file_path, "r", encoding='utf-8').readlines():
            word, index = line.strip().split("\t")
            index = int(index)
            # 如果vocab 超过了指定大小
            # 跳出循环 截断
            if vocab_max_size and index > vocab_max_size - Vocab.MASK_COUNT:
                print("max_size of vocab was specified as %i; we now have %i words. Stopping reading." % (
                    vocab_max_size, index))
                break
            vocab[word] = index + Vocab.MASK_COUNT
            reverse_vocab[index + Vocab.MASK_COUNT] = word
        return vocab, reverse_vocab

    def word_to_id(self, word):
        if word not in self.word2id:
            return self.word2id[self.UNKNOWN_TOKEN]
        return self.word2id[word]

    def id_to_word(self, word_id):
        if word_id not in self.id2word:
            raise ValueError('Id not found in vocab: %d' % word_id)
        return self.id2word[word_id]

    def size(self):
        return self.count